<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/Lab_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implement any algorithm from the lecture to solve the grid game (e.g., Q-Learning)

In [1]:
# Import necessary libraries
import numpy as np

In [8]:
# Environment Class
class Grid:
    def __init__(self, width, height, start):
        self.width = width
        self.height = height
        self.i = start[0]
        self.j = start[1]

    def set_grid(self, rewards, actions):
        # rewards should be a dict of: (i, j): r (row, col): reward
        # actions should be a dict of: (i, j): A (row, col): list of possible actions
        self.rewards = rewards
        self.actions = actions

    def set_state(self, s):
        self.i = s[0]
        self.j = s[1]

    def current_state(self):
        return (self.i, self.j)

    def is_terminal(self, s):
        return s not in self.actions

    def move(self, action):
        # check if legal move first
        if action in self.actions[(self.i, self.j)]:
            if action == 'U':
                self.i -= 1
            elif action == 'D':
                self.i += 1
            elif action == 'R':
                self.j += 1
            elif action == 'L':
                self.j -= 1
        # return a reward (if any)
        return self.rewards.get((self.i, self.j), 0)

    def game_over(self):
        # returns true if game is over, else false
        # true if we are in a state where no actions are possible
        return (self.i, self.j) not in self.actions

    def all_states(self):
        # returns either a position that has possible next actions
        # or a position that yields a reward
        return set(self.actions.keys()) | set(self.rewards.keys())

In [9]:
### CREATE GRID OBJECT INSTANCE WITH ACTIONS/REWARDS
## HERE

# Create a 2x2 grid
grid = Grid(2, 2, (0, 0))

# Define rewards and actions
rewards = {(1, 1): 1, (0, 0): 0, (0, 1): 0, (1, 0): 0}
actions = {
    (0, 0): ['D', 'R'],
    (0, 1): ['L', 'D'],
    (1, 0): ['U', 'R'],
    # (1, 1) is a terminal state, so no actions are defined
}

# Set the grid with the defined rewards and actions
grid.set_grid(rewards, actions)

In [10]:
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')
### INITIALIZE NECCESSARY DATA STRUCTS (e.g., Q table)
### HERE

import numpy as np

# Initialize Q-table
Q = {}
states = [(i, j) for i in range(2) for j in range(2)]
for s in states:
    Q[s] = {}
    for a in ALL_POSSIBLE_ACTIONS:
        Q[s][a] = 0  # initialize Q-values to 0

# Initialize returns, optional
returns = {}
for s in states:
    for a in ALL_POSSIBLE_ACTIONS:
        returns[(s, a)] = []

# Initialize state-action count, useful for updating Q-values
state_action_count = {}
for s in states:
    for a in ALL_POSSIBLE_ACTIONS:
        state_action_count[(s, a)] = 0

# Example of accessing a Q-value for state (0, 0) and action 'U'
print("Q-value for state (0, 0) and action 'U':", Q[(0, 0)]['U'])

Q-value for state (0, 0) and action 'U': 0


In [11]:
### DEFINE HELPER FUNCTIONS, IF NEEDED (e.g., random_action)
### HERE

import random

def random_action(a, eps=0.1):
    """
    Choose an action using an epsilon-greedy strategy.
    With probability epsilon, we choose a random action.
    With probability 1 - epsilon, we choose action 'a'.
    """
    p = np.random.random()
    if p < (1 - eps):
        return a
    else:
        return np.random.choice(ALL_POSSIBLE_ACTIONS)

def max_dict(d):
    """
    Return the key and value with the highest value from dictionary 'd'.
    Here 'd' is expected to be a dictionary of action: Q-value pairs.
    """
    max_key = None
    max_val = float('-inf')
    for k, v in d.items():
        if v > max_val:
            max_val = v
            max_key = k
    return max_key, max_val

def update_Q(s, a, r, s_next, alpha, gamma):
    """
    Update Q-value for a state-action pair Q(s,a).
    s: current state
    a: action taken
    r: reward received
    s_next: next state
    alpha: learning rate
    gamma: discount factor
    """
    max_a_next, max_q_next = max_dict(Q[s_next])
    Q[s][a] += alpha * (r + gamma * max_q_next - Q[s][a])

In [12]:
### SET ALGORITHM HYPERPARAMETERS
### HERE

# Learning rate
alpha = 0.1

# Discount factor
gamma = 0.9

# Epsilon for epsilon-greedy strategy
epsilon = 0.1

# Number of episodes to run the agent
num_episodes = 1000

# Maximum steps per episode (optional, depending on your environment)
max_steps_per_episode = 100

In [13]:
s = (0, 0) # start state
grid.set_state(s)

### IMPLEMENT TRAINING LOOP
### HERE

for episode in range(num_episodes):
    # Reset the state back to the starting state
    s = (0, 0)
    grid.set_state(s)

    for t in range(max_steps_per_episode):
        # Choose an action using the epsilon-greedy strategy
        a, _ = max_dict(Q[s])
        a = random_action(a, epsilon)

        # Take the action and observe the new state and reward
        r = grid.move(a)
        s_next = grid.current_state()

        # Update the Q-table
        update_Q(s, a, r, s_next, alpha, gamma)

        # Update the state
        s = s_next

        # Check if the state is terminal
        if grid.game_over():
            break

# Optional: Print final Q-values for analysis
for s in Q:
    print(s, Q[s])

(0, 0) {'U': 0.7293753486039924, 'D': 0.5360895095367246, 'L': 0.7851318499205143, 'R': 0.899999999999999}
(0, 1) {'U': 0.8757899668957252, 'D': 0.9999999999999996, 'L': 0.7719873547082745, 'R': 0.8417262113984965}
(1, 0) {'U': 0.7479857819446121, 'D': 0.05993436755383791, 'L': 0.06568760207834101, 'R': 0.1}
(1, 1) {'U': 0, 'D': 0, 'L': 0, 'R': 0}


In [14]:
### PRINT FINAL POLICY (actions learnt for each state)
### HERE

final_policy = {}
for s in Q:
    best_action, _ = max_dict(Q[s])
    final_policy[s] = best_action

# Print the final policy
print("Final Policy:")
for s in final_policy:
    action = final_policy[s]
    print("State:", s, "-> Action:", action)

Final Policy:
State: (0, 0) -> Action: R
State: (0, 1) -> Action: D
State: (1, 0) -> Action: U
State: (1, 1) -> Action: U
